In [1]:
import requests
import pandas as pd
import tqdm
from collections import defaultdict
import urllib3
import os
import csv
import random
from concurrent.futures import ThreadPoolExecutor, as_completed

In [2]:
#files = os.listdir('./input/')
#csv_files = [file for file in files if file.endswith('.csv')][:-1]
#all_data = pd.concat([pd.read_csv(f'./input/{file}') for file in csv_files])

In [3]:
#for file in csv_files:
#    df = pd.read_csv(f'./input/{file}')
#    output_file = f'./input/{file}.parquet'
#    df.to_parquet(output_file, compression='snappy')  
#    print(f'Archivo {file} guardado como {output_file}')

In [4]:
#all_data = all_data.drop_duplicates()

In [5]:
#all_data.to_parquet('./input/registro_nacional_sociedades_2do.parquet', compression = 'snappy')

In [2]:
all_data = pd.read_parquet('./input/registro_nacional_sociedades.parquet')

In [22]:
cuits = all_data['cuit'].unique()

In [5]:
base_url = "https://api.bcra.gob.ar/centraldedeudores/v1.0/Deudas/Historicas/"
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

In [23]:
todas_las_filas = []
errores = defaultdict(str)

session = requests.Session()

def fetch_data(id):
	try:
		url = f"{base_url}{id}"
		res = session.get(url, verify=False)
		res.raise_for_status()
		data = res.json()
		
		results = data['results']
		identificacion_val = results.get('identificacion')
		denominacion = results.get('denominacion')
		periodos = results.get('periodos', [])
		
		filas = []
		for periodo in periodos:
			periodo_val = periodo.get('periodo')
			entidades = periodo.get('entidades', [])
			for entidad in entidades:
				fila = {'identificacion': identificacion_val,
						'denominacion': denominacion,
						'periodo': periodo_val,
						'entidad': entidad.get('entidad'),
						'situacion': entidad.get('situacion'),
						'monto': entidad.get('monto'),
						'enRevision': entidad.get('enRevision'),
						'procesoJud': entidad.get('procesoJud')}
				filas.append(fila)
		return filas, None
	except requests.exceptions.RequestException as e:
		return None, e

with ThreadPoolExecutor() as executor:
	futures = {executor.submit(fetch_data, id): id for id in cuits}
	for future in tqdm.tqdm(as_completed(futures), total=len(futures)):
		filas, error = future.result()
		if filas:
			todas_las_filas.extend(filas)
		if error:
			errores[futures[future]] = str(error)

100%|██████████| 104899/104899 [07:17<00:00, 239.74it/s]


In [24]:
columnas = ['identificacion', 'denominacion', 'periodo', 'entidad', 'situacion', 'monto', 'enRevision', 'procesoJud']
creditos_hist = pd.DataFrame(todas_las_filas, columns= columnas)

In [19]:
creditos_hist.to_parquet('./input/creditos_hist.parquet', compression= 'gzip')

## No Históricas

In [3]:
base_url = "https://api.bcra.gob.ar/centraldedeudores/v1.0/Deudas/"
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

In [5]:
cuits = all_data['cuit'].unique()

In [ ]:
session = requests.Session()

def fetch_data(id):
	try:
		url = f"{base_url}{id}"
		res = session.get(url, verify=False)
		res.raise_for_status()
		data = res.json()
		
		results = data['results']
		identificacion_val = results.get('identificacion')
		denominacion = results.get('denominacion')
		periodos = results.get('periodos', [])
		
		filas = []
		for periodo in periodos:
			periodo_val = periodo.get('periodo')
			entidades = periodo.get('entidades', [])
			for entidad in entidades:
				fila = {'identificacion': identificacion_val,
						'denominacion': denominacion,
						'periodo': periodo_val,
						'entidad': entidad.get('entidad'),
						'situacion': entidad.get('situacion'),
						'monto': entidad.get('monto'),
						'enRevision': entidad.get('enRevision'),
						'procesoJud': entidad.get('procesoJud'),
						'fechaSit1': entidad.get('fechaSit1'),
						'diasAtrasoPago': entidad.get('diasAtrasoPago'),
						'refinanciaciones': entidad.get('refinanciaciones'),
						'recategorizacionOblig': entidad.get('recategorizacionOblig'),
						'situacionJuridica': entidad.get('situacionJuridica'),
						'irrecDisposicionTecnica': entidad.get('irrecDisposicionTecnica')}
				filas.append(fila)
		return filas, None
	except requests.exceptions.RequestException as e:
		return None, (id, str(e))

errores = defaultdict(str)
todas_las_filas = []

with ThreadPoolExecutor() as executor:
	futures = {executor.submit(fetch_data, id): id for id in cuits}
	for future in tqdm.tqdm(as_completed(futures), total=len(futures)):
		filas, error = future.result()
		if filas:
			todas_las_filas.extend(filas)
		if error:
			errores[error[0]] = error[1]

In [7]:
columnas = ['identificacion', 'denominacion', 'periodo', 'entidad', 'situacion', 'monto', 'enRevision', 'procesoJud', 'fechaSit1', 'diasAtrasoPago', 'refinanciaciones', 'recategorizacionOblig', 'situacionJuridica', 'irrecDisposicionTecnica']
creditos = pd.DataFrame(todas_las_filas, columns= columnas)

In [11]:
creditos.to_parquet('./input/creditos.parquet', compression= 'gzip')